In [5]:
# Packages
import pandas as pd
import os
import datetime as dt
import numpy as np
import pyproj

geod = pyproj.Geod(ellps='WGS84')
from multiprocessing.pool import Pool



In [6]:
# Settings (file names and directories)

LABELS_FILE = 'labels.txt'
MAIN_FOLDER = './data/'
TRAJ_FOLDER = 'Trajectory/'
OUTPUT_FOLDER = './data_preprocessed3/'
POOLSIZE = 4



In [7]:
# Distance between 2 points
def calculate_distance(long1, lat1, long2, lat2):
    if lat1 == lat2 and long1 == long2:
        return 0
    if False in np.isfinite([long1, long2, lat1, lat2]):
        return np.nan
    if lat1 < -90 or lat1 > 90 or lat2 < -90 or lat2 > 90:
        #raise ValueError('The range of latitudes seems to be invalid.')
        return np.nan
    if long1 < -180 or long1 > 180 or long2 < -180 or long2 > 180:
        return np.nan
        #raise ValueError('The ranя ge of longitudes seems to be invalid.')
    angle1,angle2,distance = geod.inv(long1, lat1, long2, lat2)
    return distance

# Get velocity distance/timedelta
def calculate_velocity(distance, timedelta):
    if timedelta.total_seconds() == 0: return np.nan
    return distance / timedelta.total_seconds()

# Get acceleration diff(velocity)/timedelta
def calculate_acceleration(velocity, velocity_next_position, timedelta):
    delta_v = velocity_next_position - velocity
    if timedelta.total_seconds() == 0: return np.nan
    return delta_v / timedelta.total_seconds()

In [8]:
headers_trajectory = ['lat', 'long', 'null', 'altitude','timestamp_float', 'date', 'time']

# Convert str datetime to datetime format
def to_datetime(string):
    return dt.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')

# Load labels in the folder
def load_labels_df(filename):
    df = pd.read_csv(filename, sep='\t')
    df['start_time'] = df['Start Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['end_time'] = df['End Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['labels'] = df['Transportation Mode']
    df = df.drop(['End Time', 'Start Time', 'Transportation Mode'], axis=1)
    return df

# Load trajectory
def load_trajectory_df(full_filename):
    subfolder = full_filename.split('/')[-3]
    trajectory_id = full_filename.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(full_filename, skiprows = 6, header = None, names = headers_trajectory)
   
    df['datetime'] = df.apply(lambda z: to_datetime(z.date + ' ' + z.time), axis=1)
    df['datetime_next_position'] = df['datetime'].shift(-1)
    df['timedelta'] = df.apply(lambda z: z.datetime_next_position - z.datetime, axis=1)
    df = df.drop(['datetime_next_position'], axis=1)
    df = df.drop(['null', 'timestamp_float', 'date', 'time'], axis=1)
    
    df['long_next_position'] = df['long'].shift(-1)
    df['lat_next_position'] = df['lat'].shift(-1)
    df['distance'] = df.apply(lambda z: calculate_distance(z.long, z.lat, z.long_next_position, z.lat_next_position), axis=1)
    df = df.drop(['long_next_position', 'lat_next_position'], axis=1)
    
    df['velocity'] = df.apply(lambda z: calculate_velocity(z.distance, z.timedelta), axis=1)
    df['velocity_next_position'] = df['velocity'].shift(-1)
    df['acceleration'] = df.apply(lambda z: calculate_acceleration(z.velocity, z.velocity_next_position, z.timedelta), axis=1)
    df = df.drop(['velocity_next_position'], axis=1)
    
    df['trajectory_id'] = trajectory_id
    df['subfolder'] = subfolder
    df['labels'] = ''
    calculate_agg_features(df)
    return df

#This method calculates the aggregated feature and 
#saves them in the original df
def calculate_agg_features(df):
    v_ave = np.nanmean(df['velocity'].values)
    v_med = np.nanmedian(df['velocity'].values)
    v_max = np.nanmax(df['velocity'].values)
    v_std = np.nanstd(df['velocity'].values)
    
    a_ave = np.nanmean(df['acceleration'].values)
    a_med = np.nanmedian(df['acceleration'].values)
    a_max = np.nanmax(df['acceleration'].values)
    a_std = np.nanstd(df['acceleration'].values)
   
    df.loc[:, 'v_ave'] = v_ave
    df.loc[:, 'v_med'] = v_med
    df.loc[:, 'v_max'] = v_max
    df.loc[:, 'v_std'] = v_std
    
    df.loc[:, 'a_ave'] = a_ave
    df.loc[:, 'a_med'] = a_med
    df.loc[:, 'a_max'] = a_max
    df.loc[:, 'a_std'] = a_std

In [ ]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
directories = os.listdir(MAIN_FOLDER)

# Garbage collector used to collect garbage 
# becouse OS handles it slow
import gc

# Go to the dirs and try load it.
# Save in the preprocessed folder
for subfolder in directories:
    list_df_traj = []
    subfolder_ = MAIN_FOLDER + subfolder + '/'
    traj_folder = MAIN_FOLDER + subfolder + '/' + TRAJ_FOLDER
    traj_files = os.listdir(traj_folder)
    
    traj_files_full_path = [traj_folder + traj_file for traj_file in traj_files]
    print(subfolder, len(traj_files_full_path))
    
    for i, file in enumerate(traj_files_full_path):
        list_df_traj.append(load_trajectory_df(file))
    
    # Get list of trajectories and append to the global df_traj_all
    # then clean it
    df_traj_all = pd.concat(list_df_traj)
    list_df_traj = []
    
    # If there label file in the folder do preprocessing
    if LABELS_FILE in os.listdir(subfolder_):
        filename = subfolder_ + LABELS_FILE
        df_labels = load_labels_df(filename)
        for idx in df_labels.index.values:
            st = df_labels.ix[idx]['start_time']
            et = df_labels.ix[idx]['end_time']
            labels = df_labels.ix[idx]['labels']
            if labels:
                df_traj_all.loc[(df_traj_all['datetime'] >= st) & 
                                (df_traj_all['datetime'] <= et), 'labels'] = labels
    
    # Save file with labels and aggegated features
    output_filename = OUTPUT_FOLDER + subfolder + '.csv'
    df_traj_all.to_csv(output_filename)
    
    # Delete file and try to erise memory with gc(garbage collector) 
    del df_traj_all
    gc.collect()
    

142 156


# Results:

### 1) All GPS and labels data converted to the one CSV file.

### 2) Created features like velocity, acceleration, data preprocessing.

### 3) Created statistics like max, std, mean, median.

Data structure before             |  Data sctructure after
:-------------------------:|:-------------------------:
![title](jupyter_images/0. Data Preparation/raw_data_folder.jpg)  |  ![title](jupyter_images/0. Data Preparation/preprocessed_data.jpg)